In [1]:
import pandas as pd
import sqlite3
from pandas.io import sql

In [2]:
transaction_table = pd.read_csv('dat/transaction_table.csv')
transaction_table.head()

,transaction_id,product_id,customer_id,store_id,transaction_date,transaction_amount
0,1,P4,C4,S4,1/1/2021,85
1,2,P1,C3,S2,1/1/2021,80
2,3,P4,C1,S1,1/1/2021,45
3,4,P4,C3,S3,1/1/2021,98
4,5,P1,C2,S1,1/1/2021,17


In [40]:
def weare():
    return 1+1

2

In [3]:
customer_table = pd.read_csv('dat/customer_table.csv')
customer_table

,customer_id,name,gender,age
0,C1,Sam,M,20
1,C2,Oliver,M,25
2,C3,Tina,F,30
3,C4,Mary,F,15


In [4]:
product_table = pd.read_csv('dat/product_table.csv')
product_table

,product_id,product_group
0,P1,Cloth
1,P2,Cloth
2,P3,Luggage
3,P4,Food


In [5]:
location_table = pd.read_csv('dat/location_table.csv')
location_table

,store_id,Neighbourhood
0,S1,Orchard
1,S2,Orchard
2,S3,Bugis
3,S4,Raffles


In [6]:
db_connection = sqlite3.connect('homework.db.sqlite')

transaction_table.to_sql('transaction_table', con = db_connection, if_exists = 'replace', index = False)
customer_table.to_sql('customer_table', con = db_connection, if_exists = 'replace', index = False)
product_table.to_sql('product_table', con = db_connection, if_exists = 'replace', index = False)
location_table.to_sql('location_table', con = db_connection, if_exists = 'replace', index = False)

In [29]:
#Which are our high end customers? Which are our loyal customers? Which customers are churning?"

query = """
        SELECT c.name as customer_name, sum(t.transaction_amount) as total_amt_spent,
        count(t.transaction_id) as number_of_transactions,
        MAX(t.transaction_date) as last_transaction,
            
        CASE 
        WHEN c.name = 'Mary' THEN 'high end / loyal'
        WHEN c.name = 'Sam' THEN 'high end / loyal'
        WHEN c.name = 'Oliver' THEN 'loyal'
        ELSE 'churning' END AS cust_status
        
        FROM transaction_table t
        LEFT JOIN customer_table c
        ON t.customer_id = c.customer_id
        GROUP by customer_name
        ORDER by total_amt_spent DESC
        """

high_end_customers = sql.read_sql(query, con=db_connection)
high_end_customers

,customer_name,total_amt_spent,number_of_transactions,last_transaction,cust_status
0,Mary,1644,29,9/1/2021,high end / loyal
1,Sam,1555,26,9/1/2021,high end / loyal
2,Oliver,1190,26,9/1/2021,loyal
3,Tina,937,19,9/1/2021,churning


In [26]:
query = """
        SELECT c.gender as gender,
        sum(CASE WHEN p.product_group = 'Cloth' THEN t.transaction_amount ELSE 0 END) AS total_cloth_spending,
        sum(CASE WHEN p.product_group = 'Luggage' THEN t.transaction_amount ELSE 0 END) AS total_luggage_spending,
        sum(CASE WHEN p.product_group = 'Food' THEN t.transaction_amount ELSE 0 END) AS total_food_spending,
        sum(transaction_amount) as total_spending
        FROM transaction_table t
        LEFT JOIN customer_table c
        ON t.customer_id = c.customer_id
        LEFT JOIN product_table p
        ON t.product_id = p.product_id
        GROUP By c.gender
        ORDER By total_spending DESC
        """

q6 = sql.read_sql(query, con=db_connection)
q6

,gender,total_cloth_spending,total_luggage_spending,total_food_spending,total_spending
0,M,1371,689,685,2745
1,F,1351,366,864,2581


In [9]:
query = """            
        SELECT COUNT(DISTINCT sub.customer_id) as count_of_customers
        FROM
            (SELECT *
            FROM transaction_table t
            LEFT JOIN location_table l
            ON t.store_id = l.store_id
            LEFT JOIN product_table p
            ON t.product_id = p.product_id
            WHERE l.Neighbourhood = 'Orchard'
            AND t.transaction_date = '7/1/2021'
            AND p.product_group = 'Cloth'
            GROUP BY t.customer_id
            HAVING sum(t.transaction_amount) >=50) sub
        """

q5 = sql.read_sql(query, con=db_connection)
q5


,count_of_customers
0,2
